In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd

options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

cafe_data = []
processed_links = set()

try:
    # 크롤하고자 하는 페이지 수
    total_pages = 5  # 예: 첫 5 페이지 크롤

    for page in range(1, total_pages + 1):
        search_url = f"https://top.cafe.daum.net/_c21_/search/cafe-table?searchOpt=CAFE_ARTICLE&articleSortType=ACCURACY&q=irp&p={page}"
        driver.get(search_url)
        time.sleep(3)

        print(f"\n=== {page} 페이지 크롤 중... ===")

        # 예를 들면 페이지 내 여러 아이템이 있으면 여기에 find_elements 로 변경
        try:
            title_element = driver.find_element(By.CSS_SELECTOR, "#articleContentWrap1 > div.scafe_cont > strong")
            content_element = driver.find_element(By.CSS_SELECTOR, "#articleContentWrap1 > div.scafe_cont > p")

            title = title_element.text.strip() if title_element else "제목 없음"
            content = content_element.text.strip() if content_element else "내용 없음"
            link = driver.current_url

            if link not in processed_links:
                processed_links.add(link)
                cafe_data.append({
                    "순번": len(cafe_data) + 1,
                    "제목": title,
                    "내용": content,
                    "링크": link
                })
                print(f"  [{page}] {title[:30]}... / {content[:30] if content else '내용 없음'}")
        except Exception as e:
            print(f"⚠️ [{page}] 아이템 추출 실패: {e}")

    # 결과 저장
    if cafe_data:
        df = pd.DataFrame(cafe_data)
        filename = f"daum_cafe_irp_crawling_{int(time.time())}.csv"
        df.to_csv(filename, encoding="utf-8-sig", index=False)
        print(f"\n✅ 결과가 '{filename}' 파일로 저장되었습니다.")
        print(f"📊 수집된 글 수: {len(cafe_data)}개")
    else:
        print("❌ 크롤된 데이터가 없습니다.")

finally:
    driver.quit()
    print("\n🔚 브라우저가 종료되었습니다.")



=== 1 페이지 크롤 중... ===
  [1] IRP 계좌 만들기 잘 아시는 분 저 퇴직금 받을 수 ... / 오늘 하루종일 붙잡고 씨름중인데 도저히 이해가 안돼서 

=== 2 페이지 크롤 중... ===
  [2] irp 계좌개설 해서 퇴직금 받고 해지하면 세금을 많이... / 이번에 퇴사하면서 처음 개설중인데 뭐가 뭔지 모르겠어요

=== 3 페이지 크롤 중... ===
  [3] irp 계좌 퇴직금 수령할때 세금 얼마떼고 받았어?? ... / 나 한 1200정도 되는데 얼마정도 때려나,, 너넨 얼

=== 4 페이지 크롤 중... ===
  [4] 혹시 irp(개인형노후연금) 운용하시는 분들 etf 추... / 개인적으로 투자하는 etf는 잇는데 irp내 운용자산을

=== 5 페이지 크롤 중... ===
  [5] 퇴직위로금은 원래 IRP로 넣어주나요?? 2024.07... / 거짓말처럼 때마침 기존회사에서 희망퇴직을 받는다하여 급

✅ 결과가 'daum_cafe_irp_crawling_1750739101.csv' 파일로 저장되었습니다.
📊 수집된 글 수: 5개

🔚 브라우저가 종료되었습니다.


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd

# 브라우저 옵션
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

results = []
try:
    total_pages = 3  # 크롤하고자 하는 페이지 수
    for page in range(1, total_pages + 1):
        url = f"https://top.cafe.daum.net/_c21_/search/cafe-table?searchOpt=CAFE_ARTICLE&articleSortType=ACCURACY&q=irp&p={page}"
        driver.get(url)
        time.sleep(3)

        print(f"\n=== {page} 페이지 크롤 중... ===")

        # 1️⃣ 페이지 내 모든 게시글 찾기
        items = driver.find_elements(By.CSS_SELECTOR, "table.tbl tr td.td_subject a.link_tit")
        links = []
        titles = []
        for item in items:
            links.append(item.get_attribute("href"))
            titles.append(item.text.strip())

        # 2️⃣ 각 게시글 링크로 접속해 제목 + 내용 추출
        for title, link in zip(titles, links):
            try:
                driver.get(link)
                time.sleep(2)

                # 제목 추출
                try:
                    post_title = driver.find_element(By.CSS_SELECTOR, "#articleContentWrap1 > div.scafe_cont > strong").text.strip()
                except:
                    post_title = title  # 예비로 원래 목록에서 가져온 제목

                # 내용 추출
                try:
                    post_content = driver.find_element(By.CSS_SELECTOR, "#articleContentWrap1 > div.scafe_cont > p").text.strip()
                except:
                    post_content = "내용 없음"

                results.append({
                    "페이지": page,
                    "제목": post_title,
                    "내용": post_content,
                    "링크": link
                })
                print(f"[{page}] ✅ {post_title[:30]}... 크롤 성공!")

            except Exception as e:
                results.append({
                    "페이지": page,
                    "제목": title,
                    "내용": "크롤 실패",
                    "링크": link
                })
                print(f"[{page}] ❌ 크롤 실패: {title}")

    # 결과 저장
    if results:
        df = pd.DataFrame(results)
        filename = f"daum_cafe_all_pages_{int(time.time())}.csv"
        df.to_csv(filename, encoding="utf-8-sig", index=False)
        print(f"\n✅ 크롤된 {len(results)}개의 게시글이 '{filename}'로 저장되었습니다.")
    else:
        print("❌ 크롤된 데이터가 없습니다.")

finally:
    driver.quit()
    print("\n🔚 브라우저가 종료되었습니다.")



=== 1 페이지 크롤 중... ===

=== 2 페이지 크롤 중... ===

=== 3 페이지 크롤 중... ===
❌ 크롤된 데이터가 없습니다.

🔚 브라우저가 종료되었습니다.


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import re

# 브라우저 옵션 설정
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
# User-Agent 설정으로 bot 감지 우회
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

results = []
wait = WebDriverWait(driver, 10)

try:
    total_pages = 3  # 크롤하고자 하는 페이지 수
    
    for page in range(1, total_pages + 1):
        url = f"https://top.cafe.daum.net/_c21_/search/cafe-table?searchOpt=CAFE_ARTICLE&articleSortType=ACCURACY&q=irp&p={page}"
        print(f"\n=== {page} 페이지 접속 중... ===")
        print(f"URL: {url}")
        
        driver.get(url)
        time.sleep(5)  # 페이지 로딩 대기 시간 증가
        
        # 페이지가 제대로 로드되었는지 확인
        try:
            # 다양한 선택자로 게시글 목록 찾기 시도
            selectors_to_try = [
                "table.tbl tr td.td_subject a.link_tit",
                ".tbl tr .td_subject a",
                "table tr td a[href*='article']",
                ".list_board .subject a",
                "a[href*='cafe.daum.net']"
            ]
            
            items = []
            for selector in selectors_to_try:
                try:
                    items = driver.find_elements(By.CSS_SELECTOR, selector)
                    if items:
                        print(f"✅ 선택자 '{selector}'로 {len(items)}개 항목 발견")
                        break
                except:
                    continue
            
            if not items:
                print(f"❌ {page} 페이지에서 게시글을 찾을 수 없습니다.")
                # 페이지 소스 일부 출력으로 디버깅
                page_source = driver.page_source[:1000]
                print(f"페이지 소스 일부: {page_source}")
                continue
            
            # 링크와 제목 수집
            links = []
            titles = []
            
            for item in items:
                try:
                    link = item.get_attribute("href")
                    title = item.text.strip()
                    
                    # 유효한 카페 링크인지 확인
                    if link and 'cafe.daum.net' in link and title:
                        links.append(link)
                        titles.append(title)
                except Exception as e:
                    continue
            
            print(f"📋 {page} 페이지에서 {len(links)}개의 유효한 게시글 발견")
            
            # 각 게시글 내용 크롤링
            for idx, (title, link) in enumerate(zip(titles, links), 1):
                try:
                    print(f"[{page}-{idx}] {title[:30]}... 크롤링 중...")
                    driver.get(link)
                    time.sleep(3)
                    
                    # 다양한 선택자로 제목 추출 시도
                    post_title = title  # 기본값
                    title_selectors = [
                        "h3.tit_view",
                        ".tit_view",
                        "h3",
                        ".subject",
                        "strong"
                    ]
                    
                    for sel in title_selectors:
                        try:
                            element = driver.find_element(By.CSS_SELECTOR, sel)
                            if element.text.strip():
                                post_title = element.text.strip()
                                break
                        except:
                            continue
                    
                    # 다양한 선택자로 내용 추출 시도
                    post_content = "내용 추출 실패"
                    content_selectors = [
                        ".article_view",
                        ".content_view", 
                        ".txt_detail",
                        "#content",
                        ".view_content",
                        "div[class*='content']",
                        "p"
                    ]
                    
                    for sel in content_selectors:
                        try:
                            elements = driver.find_elements(By.CSS_SELECTOR, sel)
                            for element in elements:
                                text = element.text.strip()
                                if len(text) > 10:  # 의미있는 내용인지 확인
                                    post_content = text[:500]  # 내용 길이 제한
                                    break
                            if post_content != "내용 추출 실패":
                                break
                        except:
                            continue
                    
                    # 결과 저장
                    results.append({
                        "페이지": page,
                        "순서": idx,
                        "제목": post_title,
                        "내용": post_content,
                        "링크": link
                    })
                    
                    print(f"[{page}-{idx}] ✅ 성공: {post_title[:30]}...")
                    
                except Exception as e:
                    print(f"[{page}-{idx}] ❌ 실패: {str(e)}")
                    results.append({
                        "페이지": page,
                        "순서": idx,
                        "제목": title,
                        "내용": f"크롤링 실패: {str(e)}",
                        "링크": link
                    })
                
                # 요청 간격 조절
                time.sleep(2)
        
        except Exception as e:
            print(f"❌ {page} 페이지 처리 중 오류 발생: {str(e)}")
    
    # 결과 저장
    if results:
        df = pd.DataFrame(results)
        filename = f"daum_cafe_irp_search_{int(time.time())}.csv"
        df.to_csv(filename, encoding="utf-8-sig", index=False)
        print(f"\n✅ 총 {len(results)}개의 게시글이 '{filename}'에 저장되었습니다.")
        
        # 결과 요약 출력
        print("\n📊 크롤링 결과 요약:")
        print(f"- 총 게시글 수: {len(results)}")
        success_count = len([r for r in results if "크롤링 실패" not in r["내용"]])
        print(f"- 성공한 크롤링: {success_count}개")
        print(f"- 실패한 크롤링: {len(results) - success_count}개")
        
    else:
        print("❌ 크롤링된 데이터가 없습니다.")
        print("다음 사항을 확인해주세요:")
        print("1. 검색어 'irp'에 대한 결과가 실제로 존재하는지")
        print("2. 네트워크 연결 상태")
        print("3. 다음 카페의 구조 변경 여부")

except Exception as e:
    print(f"❌ 전체 프로세스 중 오류 발생: {str(e)}")

finally:
    driver.quit()
    print("\n🔚 브라우저가 종료되었습니다.")

# 추가 디버깅을 위한 함수
def debug_page_structure(driver, url):
    """페이지 구조 디버깅을 위한 함수"""
    driver.get(url)
    time.sleep(3)
    
    print("=== 페이지 구조 디버깅 ===")
    print(f"현재 URL: {driver.current_url}")
    print(f"페이지 제목: {driver.title}")
    
    # 주요 태그들 확인
    tags_to_check = ['table', 'tr', 'td', 'a', 'div']
    for tag in tags_to_check:
        elements = driver.find_elements(By.TAG_NAME, tag)
        print(f"{tag} 태그 개수: {len(elements)}")
    
    # 클래스명이 포함된 요소들 찾기
    all_elements = driver.find_elements(By.XPATH, "//*[@class]")
    classes = set()
    for elem in all_elements[:50]:  # 처음 50개만 확인
        class_attr = elem.get_attribute("class")
        if class_attr:
            classes.update(class_attr.split())
    
    print(f"발견된 주요 클래스명: {list(classes)[:20]}")  # 처음 20개만 출력


=== 1 페이지 접속 중... ===
URL: https://top.cafe.daum.net/_c21_/search/cafe-table?searchOpt=CAFE_ARTICLE&articleSortType=ACCURACY&q=irp&p=1
✅ 선택자 'a[href*='cafe.daum.net']'로 58개 항목 발견
📋 1 페이지에서 42개의 유효한 게시글 발견
[1-1] 로그인... 크롤링 중...
[1-1] ✅ 성공: 로그인...
[1-2] 테이블... 크롤링 중...
[1-2] ✅ 성공: 테이블 탐색...
[1-3] IRP 계좌 만들기 잘 아시는 분 저 퇴직금 받을 수 ... 크롤링 중...
[1-3] ✅ 성공: IRP 계좌 만들기 잘 아시는 분 저 퇴직금 받을 수 ...
[1-4] https://cafe.daum.net/dotax/D8... 크롤링 중...
[1-4] ✅ 성공: https://cafe.daum.net/dotax/D8...
[1-5] 도탁스 (DOTAX)... 크롤링 중...
[1-5] ✅ 성공: 도탁스 (DOTAX)...
[1-6] 카페글 미리보기... 크롤링 중...
[1-6] ✅ 성공: 카페글 미리보기...
[1-7] IRP 계좌 출금, 해지방법... 크롤링 중...
[1-7] ✅ 성공: IRP 계좌 출금, 해지방법...
[1-8] https://cafe.daum.net/getmoney... 크롤링 중...
[1-8] ✅ 성공: https://cafe.daum.net/getmoney...
[1-9] 금융지식꿀팁... 크롤링 중...
[1-9] ✅ 성공: 금융지식꿀팁...
[1-10] 카페글 미리보기... 크롤링 중...
[1-10] ✅ 성공: 카페글 미리보기...
[1-11] ISA, IRP, 연금저축계좌로 해외간접투자 하지마세요... 크롤링 중...
[1-11] ✅ 성공: ISA, IRP, 연금저축계좌로 해외간접투자 하지마세요...
[1-12] https://cafe.daum.net/dobongba... 크롤링 중...
[1-1

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import re

# 브라우저 옵션 설정
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
# User-Agent 설정으로 bot 감지 우회
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

results = []
wait = WebDriverWait(driver, 10)

try:
    # 🚀 첫 번째 페이지만 테스트
    total_pages = 1  # 일단 1페이지만 크롤링
    
    for page in range(1, total_pages + 1):
        url = f"https://top.cafe.daum.net/_c21_/search/cafe-table?searchOpt=CAFE_ARTICLE&articleSortType=ACCURACY&q=irp&p={page}"
        print(f"\n=== {page} 페이지 접속 중... ===")
        print(f"URL: {url}")
        
        driver.get(url)
        time.sleep(5)  # 페이지 로딩 대기 시간 증가
        
        # 페이지가 제대로 로드되었는지 확인
        try:
            # 다양한 선택자로 게시글 목록 찾기 시도
            selectors_to_try = [
                "table.tbl tr td.td_subject a.link_tit",
                ".tbl tr .td_subject a",
                "table tr td a[href*='article']",
                ".list_board .subject a",
                "a[href*='cafe.daum.net']"
            ]
            
            items = []
            for selector in selectors_to_try:
                try:
                    items = driver.find_elements(By.CSS_SELECTOR, selector)
                    if items:
                        print(f"✅ 선택자 '{selector}'로 {len(items)}개 항목 발견")
                        break
                except:
                    continue
            
            if not items:
                print(f"❌ {page} 페이지에서 게시글을 찾을 수 없습니다.")
                # 페이지 소스 일부 출력으로 디버깅
                page_source = driver.page_source[:1000]
                print(f"페이지 소스 일부: {page_source}")
                continue
            
            # 링크와 제목 수집
            links = []
            titles = []
            
            for item in items:
                try:
                    link = item.get_attribute("href")
                    title = item.text.strip()
                    
                    # 유효한 카페 링크인지 확인
                    if link and 'cafe.daum.net' in link and title:
                        links.append(link)
                        titles.append(title)
                except Exception as e:
                    continue
            
            print(f"📋 {page} 페이지에서 {len(links)}개의 유효한 게시글 발견")
            
            # 처음 5개 게시글만 테스트 (시간 단축)
            test_links = links[:5]
            test_titles = titles[:5]
            print(f"⚡ 빠른 테스트를 위해 처음 5개 게시글만 크롤링합니다.")
            
            # 각 게시글 내용 크롤링
            for idx, (title, link) in enumerate(zip(test_titles, test_links), 1):
                try:
                    print(f"[{page}-{idx}] {title[:30]}... 크롤링 중...")
                    driver.get(link)
                    time.sleep(3)
                    
                    # 다양한 선택자로 제목 추출 시도
                    post_title = title  # 기본값
                    title_selectors = [
                        "h3.tit_view",
                        ".tit_view",
                        "h3",
                        ".subject",
                        "strong"
                    ]
                    
                    for sel in title_selectors:
                        try:
                            element = driver.find_element(By.CSS_SELECTOR, sel)
                            if element.text.strip():
                                post_title = element.text.strip()
                                break
                        except:
                            continue
                    
                    # 원하는 정확한 선택자로 내용 추출
                    post_content = "내용 없음"
                    
                    # 우선 정확한 선택자로 시도
                    target_selectors = [
                        "#articleContentWrap1 > div.scafe_cont > p",
                        "#articleContentWrap1 .scafe_cont p",
                        "div.scafe_cont > p",
                        ".scafe_cont p"
                    ]
                    
                    for sel in target_selectors:
                        try:
                            # 단일 요소 시도
                            element = driver.find_element(By.CSS_SELECTOR, sel)
                            text = element.text.strip()
                            if text:
                                post_content = text
                                print(f"    ✅ 내용 추출 성공 (선택자: {sel})")
                                break
                        except:
                            try:
                                # 여러 요소 시도 (여러 p 태그가 있을 경우)
                                elements = driver.find_elements(By.CSS_SELECTOR, sel)
                                if elements:
                                    all_text = ""
                                    for elem in elements:
                                        text = elem.text.strip()
                                        if text:
                                            all_text += text + "\n"
                                    if all_text.strip():
                                        post_content = all_text.strip()
                                        print(f"    ✅ 내용 추출 성공 (선택자: {sel}, {len(elements)}개 요소)")
                                        break
                            except:
                                continue
                    
                    # 위 방법이 실패하면 더 넓은 범위에서 시도
                    if post_content == "내용 없음":
                        fallback_selectors = [
                            "#articleContentWrap1 div.scafe_cont",
                            "#articleContentWrap1 .scafe_cont",
                            ".scafe_cont",
                            "#articleContentWrap1"
                        ]
                        
                        for sel in fallback_selectors:
                            try:
                                element = driver.find_element(By.CSS_SELECTOR, sel)
                                # p 태그들만 추출
                                p_elements = element.find_elements(By.TAG_NAME, "p")
                                if p_elements:
                                    content_parts = []
                                    for p in p_elements:
                                        text = p.text.strip()
                                        if text and len(text) > 5:  # 의미있는 텍스트만
                                            content_parts.append(text)
                                    if content_parts:
                                        post_content = "\n".join(content_parts)
                                        print(f"    ✅ 내용 추출 성공 (폴백 선택자: {sel})")
                                        break
                            except:
                                continue
                    
                    # 디버깅 정보 출력
                    if post_content == "내용 없음":
                        print(f"    ❌ 내용 추출 실패 - 페이지 구조 확인 필요")
                        try:
                            # 해당 영역이 존재하는지 확인
                            wrap_elem = driver.find_element(By.CSS_SELECTOR, "#articleContentWrap1")
                            print(f"    - articleContentWrap1 존재: 예")
                            
                            scafe_elem = wrap_elem.find_element(By.CSS_SELECTOR, "div.scafe_cont")
                            print(f"    - scafe_cont 존재: 예")
                            
                            p_elements = scafe_elem.find_elements(By.TAG_NAME, "p")
                            print(f"    - p 태그 개수: {len(p_elements)}")
                            
                            if p_elements:
                                for i, p in enumerate(p_elements[:3]):  # 처음 3개만 확인
                                    print(f"    - p[{i}] 내용: '{p.text.strip()[:50]}...'")
                        except Exception as debug_e:
                            print(f"    - 디버깅 중 오류: {str(debug_e)}")
                    
                    # 내용 길이 제한
                    if len(post_content) > 1000:
                        post_content = post_content[:1000] + "..."
                    
                    # 결과 저장
                    results.append({
                        "페이지": page,
                        "순서": idx,
                        "제목": post_title,
                        "내용": post_content,
                        "링크": link
                    })
                    
                    print(f"[{page}-{idx}] ✅ 성공: {post_title[:30]}...")
                    
                except Exception as e:
                    print(f"[{page}-{idx}] ❌ 실패: {str(e)}")
                    results.append({
                        "페이지": page,
                        "순서": idx,
                        "제목": title,
                        "내용": f"크롤링 실패: {str(e)}",
                        "링크": link
                    })
                
                # 요청 간격 조절
                time.sleep(2)
        
        except Exception as e:
            print(f"❌ {page} 페이지 처리 중 오류 발생: {str(e)}")
    
    # 결과 저장
    if results:
        df = pd.DataFrame(results)
        filename = f"daum_cafe_irp_search_{int(time.time())}.csv"
        df.to_csv(filename, encoding="utf-8-sig", index=False)
        print(f"\n✅ 총 {len(results)}개의 게시글이 '{filename}'에 저장되었습니다.")
        
        # 결과 요약 출력
        print("\n📊 크롤링 결과 요약:")
        print(f"- 총 게시글 수: {len(results)}")
        success_count = len([r for r in results if "크롤링 실패" not in r["내용"]])
        print(f"- 성공한 크롤링: {success_count}개")
        print(f"- 실패한 크롤링: {len(results) - success_count}개")
        
    else:
        print("❌ 크롤링된 데이터가 없습니다.")
        print("다음 사항을 확인해주세요:")
        print("1. 검색어 'irp'에 대한 결과가 실제로 존재하는지")
        print("2. 네트워크 연결 상태")
        print("3. 다음 카페의 구조 변경 여부")

except Exception as e:
    print(f"❌ 전체 프로세스 중 오류 발생: {str(e)}")

finally:
    driver.quit()
    print("\n🔚 브라우저가 종료되었습니다.")

# 추가 디버깅을 위한 함수
def debug_page_structure(driver, url):
    """페이지 구조 디버깅을 위한 함수"""
    driver.get(url)
    time.sleep(3)
    
    print("=== 페이지 구조 디버깅 ===")
    print(f"현재 URL: {driver.current_url}")
    print(f"페이지 제목: {driver.title}")
    
    # 주요 태그들 확인
    tags_to_check = ['table', 'tr', 'td', 'a', 'div']
    for tag in tags_to_check:
        elements = driver.find_elements(By.TAG_NAME, tag)
        print(f"{tag} 태그 개수: {len(elements)}")
    
    # 클래스명이 포함된 요소들 찾기
    all_elements = driver.find_elements(By.XPATH, "//*[@class]")
    classes = set()
    for elem in all_elements[:50]:  # 처음 50개만 확인
        class_attr = elem.get_attribute("class")
        if class_attr:
            classes.update(class_attr.split())
    
    print(f"발견된 주요 클래스명: {list(classes)[:20]}")  # 처음 20개만 출력


=== 1 페이지 접속 중... ===
URL: https://top.cafe.daum.net/_c21_/search/cafe-table?searchOpt=CAFE_ARTICLE&articleSortType=ACCURACY&q=irp&p=1
✅ 선택자 'a[href*='cafe.daum.net']'로 58개 항목 발견
📋 1 페이지에서 42개의 유효한 게시글 발견
⚡ 빠른 테스트를 위해 처음 5개 게시글만 크롤링합니다.
[1-1] 로그인... 크롤링 중...
    ❌ 내용 추출 실패 - 페이지 구조 확인 필요
    - 디버깅 중 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#articleContentWrap1"}
  (Session info: chrome=137.0.7151.104); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x7ff6c9e5cda5+78885]
	GetHandleVerifier [0x0x7ff6c9e5ce00+78976]
	(No symbol) [0x0x7ff6c9c19bca]
	(No symbol) [0x0x7ff6c9c70766]
	(No symbol) [0x0x7ff6c9c70a1c]
	(No symbol) [0x0x7ff6c9cc4467]
	(No symbol) [0x0x7ff6c9c98bcf]
	(No symbol) [0x0x7ff6c9cc122f]
	(No symbol) [0x0x7ff6c9c98963]
	(No symbol) [0x0x7ff6c9c616b1]
	(No symbol) [0x0x7ff6c9c62443]
	GetHand

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import re

# 브라우저 옵션 설정
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
# User-Agent 설정으로 bot 감지 우회
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

results = []
wait = WebDriverWait(driver, 10)

try:
    # 🚀 첫 번째 페이지만 테스트
    total_pages = 1  # 일단 1페이지만 크롤링
    
    for page in range(1, total_pages + 1):
        url = f"https://top.cafe.daum.net/_c21_/search/cafe-table?searchOpt=CAFE_ARTICLE&articleSortType=ACCURACY&q=irp&p={page}"
        print(f"\n=== {page} 페이지 접속 중... ===")
        print(f"URL: {url}")
        
        driver.get(url)
        time.sleep(5)  # 페이지 로딩 대기 시간 증가
        
        # 페이지가 제대로 로드되었는지 확인
        try:
            # 다양한 선택자로 게시글 목록 찾기 시도
            selectors_to_try = [
                "table.tbl tr td.td_subject a.link_tit",
                ".tbl tr .td_subject a",
                "table tr td a[href*='article']",
                ".list_board .subject a",
                "a[href*='cafe.daum.net']"
            ]
            
            items = []
            for selector in selectors_to_try:
                try:
                    items = driver.find_elements(By.CSS_SELECTOR, selector)
                    if items:
                        print(f"✅ 선택자 '{selector}'로 {len(items)}개 항목 발견")
                        break
                except:
                    continue
            
            if not items:
                print(f"❌ {page} 페이지에서 게시글을 찾을 수 없습니다.")
                # 페이지 소스 일부 출력으로 디버깅
                page_source = driver.page_source[:1000]
                print(f"페이지 소스 일부: {page_source}")
                continue
            
            # 링크와 제목 수집
            links = []
            titles = []
            
            for item in items:
                try:
                    link = item.get_attribute("href")
                    title = item.text.strip()
                    
                    # 유효한 카페 링크인지 확인
                    if link and 'cafe.daum.net' in link and title:
                        links.append(link)
                        titles.append(title)
                except Exception as e:
                    continue
            
            print(f"📋 {page} 페이지에서 {len(links)}개의 유효한 게시글 발견")
            
            # 처음 5개 게시글만 테스트 (시간 단축)
            test_links = links[:5]
            test_titles = titles[:5]
            print(f"⚡ 빠른 테스트를 위해 처음 5개 게시글만 크롤링합니다.")
            
            # 각 게시글 내용 크롤링
            for idx, (title, link) in enumerate(zip(test_titles, test_links), 1):
                try:
                    print(f"[{page}-{idx}] {title[:30]}... 크롤링 중...")
                    driver.get(link)
                    time.sleep(3)
                    
                    # 다양한 선택자로 제목 추출 시도
                    post_title = title  # 기본값
                    title_selectors = [
                        "h3.tit_view",
                        ".tit_view",
                        "h3",
                        ".subject",
                        "strong"
                    ]
                    
                    for sel in title_selectors:
                        try:
                            element = driver.find_element(By.CSS_SELECTOR, sel)
                            if element.text.strip():
                                post_title = element.text.strip()
                                break
                        except:
                            continue
                    
                    # 🔍 상세 디버깅: 실제 페이지 구조 확인
                    print(f"    📋 현재 페이지 URL: {driver.current_url}")
                    print(f"    📋 페이지 제목: {driver.title}")
                    
                    # 1단계: #articleContentWrap1 존재 확인
                    try:
                        wrap1 = driver.find_element(By.CSS_SELECTOR, "#articleContentWrap1")
                        print(f"    ✅ #articleContentWrap1 존재함")
                        
                        # 2단계: div.scafe_cont 존재 확인  
                        try:
                            scafe_div = wrap1.find_element(By.CSS_SELECTOR, "div.scafe_cont")
                            print(f"    ✅ div.scafe_cont 존재함")
                            
                            # 3단계: p 태그들 확인
                            p_tags = scafe_div.find_elements(By.TAG_NAME, "p")
                            print(f"    📝 div.scafe_cont 안의 p 태그 개수: {len(p_tags)}")
                            
                            if len(p_tags) > 0:
                                post_content = ""
                                for i, p in enumerate(p_tags):
                                    p_text = p.text.strip()
                                    print(f"    📄 p[{i}] 내용: '{p_text[:100]}{'...' if len(p_text) > 100 else ''}'")
                                    if p_text:
                                        post_content += p_text + "\n"
                                
                                if post_content.strip():
                                    post_content = post_content.strip()
                                    print(f"    ✅ 최종 내용 추출 성공! (길이: {len(post_content)}자)")
                                else:
                                    post_content = "p 태그는 있지만 내용이 비어있음"
                                    print(f"    ⚠️ p 태그는 있지만 텍스트가 비어있음")
                            else:
                                post_content = "div.scafe_cont 안에 p 태그 없음"
                                print(f"    ❌ div.scafe_cont 안에 p 태그가 없음")
                                
                                # div.scafe_cont의 전체 내용 확인
                                scafe_text = scafe_div.text.strip()
                                print(f"    📄 div.scafe_cont 전체 텍스트: '{scafe_text[:200]}{'...' if len(scafe_text) > 200 else ''}'")
                                
                                # div.scafe_cont 안의 모든 자식 요소 확인
                                all_children = scafe_div.find_elements(By.XPATH, "./*")
                                print(f"    🔍 div.scafe_cont의 자식 요소들:")
                                for child in all_children[:5]:  # 처음 5개만
                                    tag_name = child.tag_name
                                    class_name = child.get_attribute("class") or "없음"
                                    child_text = child.text.strip()[:50]
                                    print(f"        - {tag_name} (class: {class_name}): '{child_text}'")
                        
                        except Exception as scafe_error:
                            print(f"    ❌ div.scafe_cont 찾기 실패: {str(scafe_error)}")
                            
                            # div.scafe_cont가 없다면 다른 클래스명들 확인
                            print(f"    🔍 #articleContentWrap1 안의 모든 div 확인:")
                            all_divs = wrap1.find_elements(By.TAG_NAME, "div")
                            for div in all_divs[:3]:  # 처음 3개만
                                div_class = div.get_attribute("class") or "클래스없음"
                                div_text = div.text.strip()[:50]
                                print(f"        - div (class: {div_class}): '{div_text}'")
                            
                            post_content = f"div.scafe_cont 찾기 실패: {str(scafe_error)}"
                    
                    except Exception as wrap_error:
                        print(f"    ❌ #articleContentWrap1 찾기 실패: {str(wrap_error)}")
                        
                        # 비슷한 ID나 클래스 찾기
                        print(f"    🔍 비슷한 요소들 찾기:")
                        similar_elements = driver.find_elements(By.XPATH, "//*[contains(@id, 'article') or contains(@class, 'content')]")
                        for elem in similar_elements[:3]:
                            elem_id = elem.get_attribute("id") or "ID없음"
                            elem_class = elem.get_attribute("class") or "클래스없음"
                            print(f"        - ID: {elem_id}, Class: {elem_class}")
                        
                        post_content = f"#articleContentWrap1 찾기 실패: {str(wrap_error)}"
                    
                    # 결과 저장
                    results.append({
                        "페이지": page,
                        "순서": idx,
                        "제목": post_title,
                        "내용": post_content,
                        "링크": link
                    })
                    
                    print(f"[{page}-{idx}] ✅ 성공: {post_title[:30]}...")
                    
                except Exception as e:
                    print(f"[{page}-{idx}] ❌ 실패: {str(e)}")
                    results.append({
                        "페이지": page,
                        "순서": idx,
                        "제목": title,
                        "내용": f"크롤링 실패: {str(e)}",
                        "링크": link
                    })
                
                # 요청 간격 조절
                time.sleep(2)
        
        except Exception as e:
            print(f"❌ {page} 페이지 처리 중 오류 발생: {str(e)}")
    
    # 결과 저장
    if results:
        df = pd.DataFrame(results)
        filename = f"daum_cafe_irp_search_{int(time.time())}.csv"
        df.to_csv(filename, encoding="utf-8-sig", index=False)
        print(f"\n✅ 총 {len(results)}개의 게시글이 '{filename}'에 저장되었습니다.")
        
        # 결과 요약 출력
        print("\n📊 크롤링 결과 요약:")
        print(f"- 총 게시글 수: {len(results)}")
        success_count = len([r for r in results if "크롤링 실패" not in r["내용"]])
        print(f"- 성공한 크롤링: {success_count}개")
        print(f"- 실패한 크롤링: {len(results) - success_count}개")
        
    else:
        print("❌ 크롤링된 데이터가 없습니다.")
        print("다음 사항을 확인해주세요:")
        print("1. 검색어 'irp'에 대한 결과가 실제로 존재하는지")
        print("2. 네트워크 연결 상태")
        print("3. 다음 카페의 구조 변경 여부")

except Exception as e:
    print(f"❌ 전체 프로세스 중 오류 발생: {str(e)}")

finally:
    driver.quit()
    print("\n🔚 브라우저가 종료되었습니다.")

# 추가 디버깅을 위한 함수
def debug_page_structure(driver, url):
    """페이지 구조 디버깅을 위한 함수"""
    driver.get(url)
    time.sleep(3)
    
    print("=== 페이지 구조 디버깅 ===")
    print(f"현재 URL: {driver.current_url}")
    print(f"페이지 제목: {driver.title}")
    
    # 주요 태그들 확인
    tags_to_check = ['table', 'tr', 'td', 'a', 'div']
    for tag in tags_to_check:
        elements = driver.find_elements(By.TAG_NAME, tag)
        print(f"{tag} 태그 개수: {len(elements)}")
    
    # 클래스명이 포함된 요소들 찾기
    all_elements = driver.find_elements(By.XPATH, "//*[@class]")
    classes = set()
    for elem in all_elements[:50]:  # 처음 50개만 확인
        class_attr = elem.get_attribute("class")
        if class_attr:
            classes.update(class_attr.split())
    
    print(f"발견된 주요 클래스명: {list(classes)[:20]}")  # 처음 20개만 출력


=== 1 페이지 접속 중... ===
URL: https://top.cafe.daum.net/_c21_/search/cafe-table?searchOpt=CAFE_ARTICLE&articleSortType=ACCURACY&q=irp&p=1
✅ 선택자 'a[href*='cafe.daum.net']'로 58개 항목 발견
📋 1 페이지에서 42개의 유효한 게시글 발견
⚡ 빠른 테스트를 위해 처음 5개 게시글만 크롤링합니다.
[1-1] 로그인... 크롤링 중...
    📋 현재 페이지 URL: https://accounts.kakao.com/login/?continue=https%3A%2F%2Ftop.cafe.daum.net%2F_c21_%2Fsearch%2Fcafe-table%3FsearchOpt%3DCAFE_ARTICLE%26articleSortType%3DACCURACY%26q%3Dirp%26p%3D1#login
    📋 페이지 제목: 카카오계정
    ❌ #articleContentWrap1 찾기 실패: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#articleContentWrap1"}
  (Session info: chrome=137.0.7151.104); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x7ff6c9e5cda5+78885]
	GetHandleVerifier [0x0x7ff6c9e5ce00+78976]
	(No symbol) [0x0x7ff6c9c19bca]
	(No symbol) [0x0x7ff6c9c70766]
	(No symbol) [0x0x7ff

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd

options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

cafe_data = []
processed_links = set()

try:
    # 크롤하고자 하는 페이지 수
    total_pages = 5  # 예: 첫 5 페이지 크롤

    for page in range(1, total_pages + 1):
        search_url = f"https://top.cafe.daum.net/_c21_/search/cafe-table?searchOpt=CAFE_ARTICLE&articleSortType=ACCURACY&q=irp&p={page}"
        driver.get(search_url)
        time.sleep(3)

        print(f"\n=== {page} 페이지 크롤 중... ===")

        # 🔄 페이지 내 여러 게시글 링크들 찾기
        try:
            # 검색 결과 페이지에서 각 게시글 링크들 수집
            article_links = driver.find_elements(By.CSS_SELECTOR, "table.tbl tr td.td_subject a.link_tit")
            
            if not article_links:
                print(f"⚠️ [{page}] 페이지에서 게시글 링크를 찾을 수 없습니다.")
                continue
            
            print(f"📋 [{page}] 페이지에서 {len(article_links)}개의 게시글 발견")
            
            # 각 게시글 링크와 제목 수집
            links_and_titles = []
            for link_elem in article_links:
                try:
                    href = link_elem.get_attribute("href")
                    title_preview = link_elem.text.strip()
                    if href and title_preview:
                        links_and_titles.append((href, title_preview))
                except Exception as e:
                    print(f"    ⚠️ 링크 수집 중 오류: {e}")
                    continue
            
            # 각 게시글 페이지에 접속해서 상세 내용 크롤링
            for idx, (article_url, title_preview) in enumerate(links_and_titles, 1):
                if article_url in processed_links:
                    print(f"  [{page}-{idx}] 이미 처리된 링크 건너뜀")
                    continue
                
                try:
                    print(f"  [{page}-{idx}] {title_preview[:30]}... 크롤링 중")
                    driver.get(article_url)
                    time.sleep(2)
                    
                    # 🎯 개별 게시글 페이지에서 제목과 내용 추출
                    title = "제목 없음"
                    content = "내용 없음"
                    
                    # 제목 추출 시도
                    try:
                        title_element = driver.find_element(By.CSS_SELECTOR, "#articleContentWrap1 > div.scafe_cont > strong")
                        title = title_element.text.strip() if title_element else title_preview
                    except:
                        title = title_preview  # 실패하면 미리보기 제목 사용
                    
                    # 내용 추출 시도 (여러 p 태그 가능성 고려)
                    try:
                        # 먼저 scafe_cont 영역 찾기
                        scafe_cont = driver.find_element(By.CSS_SELECTOR, "#articleContentWrap1 > div.scafe_cont")
                        
                        # scafe_cont 안의 모든 p 태그들 찾기
                        content_elements = scafe_cont.find_elements(By.TAG_NAME, "p")
                        
                        if content_elements:
                            content_parts = []
                            for p_elem in content_elements:
                                p_text = p_elem.text.strip()
                                if p_text and len(p_text) > 3:  # 의미있는 텍스트만
                                    content_parts.append(p_text)
                            
                            content = "\n".join(content_parts) if content_parts else "내용 없음"
                        else:
                            # p 태그가 없다면 전체 텍스트 가져오기
                            content = scafe_cont.text.strip() or "내용 없음"
                            
                    except Exception as content_error:
                        print(f"    ⚠️ 내용 추출 실패: {content_error}")
                        content = "내용 추출 실패"
                    
                    # 데이터 저장
                    processed_links.add(article_url)
                    cafe_data.append({
                        "순번": len(cafe_data) + 1,
                        "페이지": page,
                        "게시글순서": idx,
                        "제목": title,
                        "내용": content[:500] + ("..." if len(content) > 500 else ""),  # 내용 길이 제한
                        "링크": article_url
                    })
                    
                    print(f"    ✅ [{page}-{idx}] 성공: {title[:20]}... | {content[:30] if content else '내용없음'}...")
                    
                except Exception as e:
                    print(f"    ❌ [{page}-{idx}] 게시글 처리 실패: {e}")
                    # 실패한 경우에도 기본 정보는 저장
                    cafe_data.append({
                        "순번": len(cafe_data) + 1,
                        "페이지": page,
                        "게시글순서": idx,
                        "제목": title_preview,
                        "내용": f"크롤링 실패: {str(e)}",
                        "링크": article_url
                    })
                
                # 요청 간격 조절 (차단 방지)
                time.sleep(1)
                
        except Exception as page_error:
            print(f"❌ [{page}] 페이지 처리 중 전체 오류: {page_error}")

    # 결과 저장
    if cafe_data:
        df = pd.DataFrame(cafe_data)
        filename = f"daum_cafe_irp_crawling_{int(time.time())}.csv"
        df.to_csv(filename, encoding="utf-8-sig", index=False)
        print(f"\n✅ 결과가 '{filename}' 파일로 저장되었습니다.")
        print(f"📊 수집된 글 수: {len(cafe_data)}개")
        
        # 간단한 통계
        success_count = len([item for item in cafe_data if "크롤링 실패" not in item["내용"]])
        print(f"📈 성공: {success_count}개, 실패: {len(cafe_data) - success_count}개")
        
    else:
        print("❌ 크롤된 데이터가 없습니다.")

finally:
    driver.quit()
    print("\n🔚 브라우저가 종료되었습니다.")


=== 1 페이지 크롤 중... ===
⚠️ [1] 페이지에서 게시글 링크를 찾을 수 없습니다.

=== 2 페이지 크롤 중... ===
⚠️ [2] 페이지에서 게시글 링크를 찾을 수 없습니다.

=== 3 페이지 크롤 중... ===
⚠️ [3] 페이지에서 게시글 링크를 찾을 수 없습니다.

=== 4 페이지 크롤 중... ===
⚠️ [4] 페이지에서 게시글 링크를 찾을 수 없습니다.

=== 5 페이지 크롤 중... ===
⚠️ [5] 페이지에서 게시글 링크를 찾을 수 없습니다.
❌ 크롤된 데이터가 없습니다.

🔚 브라우저가 종료되었습니다.


In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd

options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

cafe_data = []
processed_links = set()

try:
    # 크롤하고자 하는 페이지 수
    total_pages = 5  # 예: 첫 5 페이지 크롤

    for page in range(1, total_pages + 1):
        search_url = f"https://top.cafe.daum.net/_c21_/search/cafe-table?searchOpt=CAFE_ARTICLE&articleSortType=ACCURACY&q=irp&p={page}"
        driver.get(search_url)
        time.sleep(3)

        print(f"\n=== {page} 페이지 크롤 중... ===")

        # 🔄 페이지 내 여러 게시글 링크들 찾기
        try:
            # 검색 결과 페이지에서 각 게시글 링크들 수집
            article_links = driver.find_elements(By.CSS_SELECTOR, "table.tbl tr td.td_subject a.link_tit")
            
            if not article_links:
                print(f"⚠️ [{page}] 페이지에서 게시글 링크를 찾을 수 없습니다.")
                continue
            
            print(f"📋 [{page}] 페이지에서 {len(article_links)}개의 게시글 발견")
            
            # 각 게시글 링크와 제목 수집
            links_and_titles = []
            for link_elem in article_links:
                try:
                    href = link_elem.get_attribute("href")
                    title_preview = link_elem.text.strip()
                    if href and title_preview:
                        links_and_titles.append((href, title_preview))
                except Exception as e:
                    print(f"    ⚠️ 링크 수집 중 오류: {e}")
                    continue
            
            # 각 게시글 페이지에 접속해서 상세 내용 크롤링
            for idx, (article_url, title_preview) in enumerate(links_and_titles, 1):
                if article_url in processed_links:
                    print(f"  [{page}-{idx}] 이미 처리된 링크 건너뜀")
                    continue
                
                try:
                    print(f"  [{page}-{idx}] {title_preview[:30]}... 크롤링 중")
                    driver.get(article_url)
                    time.sleep(2)
                    
                    # 🎯 개별 게시글 페이지에서 제목과 내용 추출
                    title = "제목 없음"
                    content = "내용 없음"
                    
                    # 제목 추출 시도
                    try:
                        title_element = driver.find_element(By.CSS_SELECTOR, "#articleContentWrap1 > div.scafe_cont > strong")
                        title = title_element.text.strip() if title_element else title_preview
                    except:
                        title = title_preview  # 실패하면 미리보기 제목 사용
                    
                    # 내용 추출 시도 (여러 p 태그 가능성 고려)
                    try:
                        # 먼저 scafe_cont 영역 찾기
                        scafe_cont = driver.find_element(By.CSS_SELECTOR, "#articleContentWrap1 > div.scafe_cont")
                        
                        # scafe_cont 안의 모든 p 태그들 찾기
                        content_elements = scafe_cont.find_elements(By.TAG_NAME, "p")
                        
                        if content_elements:
                            content_parts = []
                            for p_elem in content_elements:
                                p_text = p_elem.text.strip()
                                if p_text and len(p_text) > 3:  # 의미있는 텍스트만
                                    content_parts.append(p_text)
                            
                            content = "\n".join(content_parts) if content_parts else "내용 없음"
                        else:
                            # p 태그가 없다면 전체 텍스트 가져오기
                            content = scafe_cont.text.strip() or "내용 없음"
                            
                    except Exception as content_error:
                        print(f"    ⚠️ 내용 추출 실패: {content_error}")
                        content = "내용 추출 실패"
                    
                    # 데이터 저장
                    processed_links.add(article_url)
                    cafe_data.append({
                        "순번": len(cafe_data) + 1,
                        "페이지": page,
                        "게시글순서": idx,
                        "제목": title,
                        "내용": content[:500] + ("..." if len(content) > 500 else ""),  # 내용 길이 제한
                        "링크": article_url
                    })
                    
                    print(f"    ✅ [{page}-{idx}] 성공: {title[:20]}... | {content[:30] if content else '내용없음'}...")
                    
                except Exception as e:
                    print(f"    ❌ [{page}-{idx}] 게시글 처리 실패: {e}")
                    # 실패한 경우에도 기본 정보는 저장
                    cafe_data.append({
                        "순번": len(cafe_data) + 1,
                        "페이지": page,
                        "게시글순서": idx,
                        "제목": title_preview,
                        "내용": f"크롤링 실패: {str(e)}",
                        "링크": article_url
                    })
                
                # 요청 간격 조절 (차단 방지)
                time.sleep(1)
                
        except Exception as page_error:
            print(f"❌ [{page}] 페이지 처리 중 전체 오류: {page_error}")

    # 결과 저장
    if cafe_data:
        df = pd.DataFrame(cafe_data)
        filename = f"daum_cafe_irp_crawling_{int(time.time())}.csv"
        df.to_csv(filename, encoding="utf-8-sig", index=False)
        print(f"\n✅ 결과가 '{filename}' 파일로 저장되었습니다.")
        print(f"📊 수집된 글 수: {len(cafe_data)}개")
        
        # 간단한 통계
        success_count = len([item for item in cafe_data if "크롤링 실패" not in item["내용"]])
        print(f"📈 성공: {success_count}개, 실패: {len(cafe_data) - success_count}개")
        
    else:
        print("❌ 크롤된 데이터가 없습니다.")

finally:
    driver.quit()
    print("\n🔚 브라우저가 종료되었습니다.")


=== 1 페이지 크롤 중... ===
⚠️ [1] 페이지에서 게시글 링크를 찾을 수 없습니다.

=== 2 페이지 크롤 중... ===
⚠️ [2] 페이지에서 게시글 링크를 찾을 수 없습니다.

=== 3 페이지 크롤 중... ===
⚠️ [3] 페이지에서 게시글 링크를 찾을 수 없습니다.

=== 4 페이지 크롤 중... ===
⚠️ [4] 페이지에서 게시글 링크를 찾을 수 없습니다.

=== 5 페이지 크롤 중... ===
⚠️ [5] 페이지에서 게시글 링크를 찾을 수 없습니다.
❌ 크롤된 데이터가 없습니다.

🔚 브라우저가 종료되었습니다.


In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd

options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

cafe_data = []
processed_links = set()

try:
    # 크롤하고자 하는 페이지 수
    total_pages =1   # 예: 첫 5 페이지 크롤

    for page in range(1, total_pages + 1):
        search_url = f"https://top.cafe.daum.net/_c21_/search/cafe-table?searchOpt=CAFE_ARTICLE&articleSortType=ACCURACY&q=irp&p={page}"
        driver.get(search_url)
        time.sleep(3)

        print(f"\n=== {page} 페이지 크롤 중... ===")

        # 🔄 페이지 내 여러 게시글 링크들 찾기
        try:
            # 🔍 실제 페이지 구조 확인
            print(f"    🔍 페이지 제목: {driver.title}")
            print(f"    🔍 현재 URL: {driver.current_url}")
            
            # 다양한 선택자로 게시글 링크 찾기 시도
            selectors_to_try = [
                "table.tbl tr td.td_subject a.link_tit",  # 원래 시도한 선택자
                "table tr .td_subject a",
                ".tbl tr td a",
                "table a[href*='cafe.daum.net']",
                "a[href*='cafe.daum.net']",
                ".subject a",
                "td a",
                "tr a",
                ".link_tit",
                "a[href*='article']"
            ]
            
            article_links = []
            successful_selector = None
            
            for selector in selectors_to_try:
                try:
                    links = driver.find_elements(By.CSS_SELECTOR, selector)
                    # 카페 게시글 링크인지 확인
                    valid_links = []
                    for link in links:
                        href = link.get_attribute("href")
                        if href and 'cafe.daum.net' in href:
                            valid_links.append(link)
                    
                    if valid_links:
                        article_links = valid_links
                        successful_selector = selector
                        print(f"    ✅ 선택자 '{selector}'로 {len(article_links)}개 링크 발견!")
                        break
                        
                except Exception as e:
                    continue
            
            # 여전히 링크를 못 찾았다면 페이지 구조 분석
            if not article_links:
                print(f"    🔍 페이지 구조 분석 중...")
                
                # 모든 a 태그 확인
                all_links = driver.find_elements(By.TAG_NAME, "a")
                print(f"    📊 전체 a 태그 개수: {len(all_links)}")
                
                # cafe.daum.net 포함된 링크들 찾기
                cafe_links = []
                for link in all_links[:20]:  # 처음 20개만 확인
                    href = link.get_attribute("href")
                    text = link.text.strip()
                    if href and 'cafe.daum.net' in href:
                        cafe_links.append((href, text))
                
                if cafe_links:
                    print(f"    🎯 cafe.daum.net 링크 {len(cafe_links)}개 발견:")
                    for i, (href, text) in enumerate(cafe_links[:5]):
                        print(f"        [{i+1}] {text[:30]}... -> {href}")
                    
                    # 이 링크들을 사용
                    article_links = [driver.find_element(By.XPATH, f"//a[@href='{href}']") for href, _ in cafe_links]
                    print(f"    ✅ 대안 방법으로 {len(article_links)}개 링크 확보!")
                else:
                    print(f"    ❌ cafe.daum.net 링크를 전혀 찾을 수 없습니다.")
                    
                    # 테이블 구조 확인
                    tables = driver.find_elements(By.TAG_NAME, "table")
                    print(f"    📊 테이블 개수: {len(tables)}")
                    
                    if tables:
                        for i, table in enumerate(tables[:3]):
                            rows = table.find_elements(By.TAG_NAME, "tr")
                            print(f"        테이블[{i}]: {len(rows)}개 행")
                    
                    continue
            
            if not article_links:
                print(f"⚠️ [{page}] 페이지에서 게시글 링크를 찾을 수 없습니다.")
                continue
            
            print(f"📋 [{page}] 페이지에서 {len(article_links)}개의 게시글 발견")
            
            # 각 게시글 링크와 제목 수집
            links_and_titles = []
            for link_elem in article_links:
                try:
                    href = link_elem.get_attribute("href")
                    title_preview = link_elem.text.strip()
                    if href and title_preview:
                        links_and_titles.append((href, title_preview))
                except Exception as e:
                    print(f"    ⚠️ 링크 수집 중 오류: {e}")
                    continue
            
            # 각 게시글 페이지에 접속해서 상세 내용 크롤링
            for idx, (article_url, title_preview) in enumerate(links_and_titles, 1):
                if article_url in processed_links:
                    print(f"  [{page}-{idx}] 이미 처리된 링크 건너뜀")
                    continue
                
                try:
                    print(f"  [{page}-{idx}] {title_preview[:30]}... 크롤링 중")
                    driver.get(article_url)
                    time.sleep(2)
                    
                    # 🎯 개별 게시글 페이지에서 제목과 내용 추출
                    title = "제목 없음"
                    content = "내용 없음"
                    
                    # 제목 추출 시도
                    try:
                        title_element = driver.find_element(By.CSS_SELECTOR, "#articleContentWrap1 > div.scafe_cont > strong")
                        title = title_element.text.strip() if title_element else title_preview
                    except:
                        title = title_preview  # 실패하면 미리보기 제목 사용
                    
                    # 내용 추출 시도 (여러 p 태그 가능성 고려)
                    try:
                        # 먼저 scafe_cont 영역 찾기
                        scafe_cont = driver.find_element(By.ID, "user_contents")
                        
                        # scafe_cont 안의 모든 p 태그들 찾기
                        content_elements = scafe_cont.find_elements(By.TAG_NAME, "p")
                        
                        if content_elements:
                            content_parts = []
                            for p_elem in content_elements:
                                p_text = p_elem.text.strip()
                                if p_text and len(p_text) > 3:  # 의미있는 텍스트만
                                    content_parts.append(p_text)
                            
                            content = "\n".join(content_parts) if content_parts else "내용 없음"
                        else:
                            # p 태그가 없다면 전체 텍스트 가져오기
                            content = scafe_cont.text.strip() or "내용 없음"
                            
                    except Exception as content_error:
                        print(f"    ⚠️ 내용 추출 실패: {content_error}")
                        content = "내용 추출 실패"
                    
                    # 데이터 저장
                    processed_links.add(article_url)
                    cafe_data.append({
                        "순번": len(cafe_data) + 1,
                        "페이지": page,
                        "게시글순서": idx,
                        "제목": title,
                        "내용": content[:500] + ("..." if len(content) > 500 else ""),  # 내용 길이 제한
                        "링크": article_url
                    })
                    
                    print(f"    ✅ [{page}-{idx}] 성공: {title[:20]}... | {content[:30] if content else '내용없음'}...")
                    
                except Exception as e:
                    print(f"    ❌ [{page}-{idx}] 게시글 처리 실패: {e}")
                    # 실패한 경우에도 기본 정보는 저장
                    cafe_data.append({
                        "순번": len(cafe_data) + 1,
                        "페이지": page,
                        "게시글순서": idx,
                        "제목": title_preview,
                        "내용": f"크롤링 실패: {str(e)}",
                        "링크": article_url
                    })
                
                # 요청 간격 조절 (차단 방지)
                time.sleep(1)
                
        except Exception as page_error:
            print(f"❌ [{page}] 페이지 처리 중 전체 오류: {page_error}")

    # 결과 저장
    if cafe_data:
        df = pd.DataFrame(cafe_data)
        filename = f"daum_cafe_irp_crawling_{int(time.time())}.csv"
        df.to_csv(filename, encoding="utf-8-sig", index=False)
        print(f"\n✅ 결과가 '{filename}' 파일로 저장되었습니다.")
        print(f"📊 수집된 글 수: {len(cafe_data)}개")
        
        # 간단한 통계
        success_count = len([item for item in cafe_data if "크롤링 실패" not in item["내용"]])
        print(f"📈 성공: {success_count}개, 실패: {len(cafe_data) - success_count}개")
        
    else:
        print("❌ 크롤된 데이터가 없습니다.")

finally:
    driver.quit()
    print("\n🔚 브라우저가 종료되었습니다.")


=== 1 페이지 크롤 중... ===
    🔍 페이지 제목: irp - Daum 카페 검색
    🔍 현재 URL: https://top.cafe.daum.net/_c21_/search/cafe-table?searchOpt=CAFE_ARTICLE&articleSortType=ACCURACY&q=irp&p=1
    ✅ 선택자 'a[href*='cafe.daum.net']'로 58개 링크 발견!
📋 [1] 페이지에서 58개의 게시글 발견
  [1-1] 로그인... 크롤링 중
    ⚠️ 내용 추출 실패: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="user_contents"]"}
  (Session info: chrome=137.0.7151.104); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x7ff6c9e5cda5+78885]
	GetHandleVerifier [0x0x7ff6c9e5ce00+78976]
	(No symbol) [0x0x7ff6c9c19bca]
	(No symbol) [0x0x7ff6c9c70766]
	(No symbol) [0x0x7ff6c9c70a1c]
	(No symbol) [0x0x7ff6c9cc4467]
	(No symbol) [0x0x7ff6c9c98bcf]
	(No symbol) [0x0x7ff6c9cc122f]
	(No symbol) [0x0x7ff6c9c98963]
	(No symbol) [0x0x7ff6c9c616b1]
	(No symbol) [0x0x7ff6c9c62443]
	GetHandleVerifier [0x0x7ff

In [ ]:

# 셀레늄의 webdriver 모듈을 임포트합니다.
from selenium import webdriver
# Chrome의 옵션을 설정하기 위한 모듈을 임포트합니다.
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
# time 모듈을 임포트합니다 (sleep 함수 사용을 위해).
import time

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# Chrome 브라우저 옵션 객체를 생성합니다.
options = Options()
# 브라우저 창을 시작할 때 최대화 상태로 시작하도록 옵션을 추가합니다.
options.add_argument("start-maximized")
# Chrome 웹드라이버를 초기화하고, 위에서 정의한 옵션을 적용합니다.
driver = webdriver.Chrome(options=options)

# url 주소 이동
driver.get("https://naver.com/")

# '증권' 클릭하기
# CSS 셀렉터로 입력 필드를 찾습니다. 'css_selector'를 복사해 오세요.
selector = '#shortcutArea > ul > li:nth-child(6) > a'
my_button = driver.find_element(By.CSS_SELECTOR, selector)
my_button.click()

# 새로 열린 마지막 탭으로 바꾸기
driver.switch_to.window(driver.window_handles[-1])

food_stock_list=[]

codes=['004370','003230','007310','271560','001680','000080','280360','005300','005180','049770','005610','017810']

for code in codes:
    url=f'https://finance.naver.com/item/sise.naver?code={code}'
    driver.get(url)
    time.sleep(1)

    # 새로 열린 마지막 탭으로 바꾸기
    driver.switch_to.window(driver.window_handles[-1])
    
    #스크롤 2번 내리기
    for i in range(0,2):
        driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        
    time.sleep(3)
    
    
    # 회사명 가져오기
    company_selector = '#middle > div.h_company > div.wrap_company > h2 > a'
    company_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, company_selector)))
    company = company_element.text
    print(f"🧑🏻‍💼 회사명: {company}")
    
    # 회사 코드 가져오기
    company_code_selector = '#middle > div.h_company > div.wrap_company > div > span'
    company_code_element = driver.find_element(By.CSS_SELECTOR, company_code_selector) 
    
    company_code = company_code_element.text  
    print(f"🔢 회사코드: {company_code}")
    
    # 시가 총액 데이터 가져오기
    total_price_selector = 'tab_con1 > div.first > table > tbody > tr.strong'
    total_price_element = driver.find_element(By.CSS_SELECTOR, '#_market_sum')

    total_price = total_price_element.text  
    print(f"💰 시총: {total_price}")

    
    # 날짜 가져오기
    iframe = driver.find_element(By.CSS_SELECTOR, "#content > div.section.inner_sub > iframe:nth-child(4)")
    driver.switch_to.frame(iframe)
    time.sleep(1)

    date = driver.find_element(By.CSS_SELECTOR, "body > table >tbody > tr:nth-child(3) > td").text
    print(f"📆 날짜: {date}")
    time.sleep(1)
    
    driver.switch_to.default_content()
    time.sleep(1)
        
    # 종가 데이터 가져오기
    price_selector = "body > table >tbody > tr:nth-child(3) > td:nth-child(2)"
    price_element = driver.find_element(By.CSS_SELECTOR, 'span.tah.p11') 
    
    price = price_element.text  
    print(f"💰 종가: {price}")
    print()
    
    
    
    doc = {
        "날짜":date,
        "회사명":company,
        "회사코드":company_code,
        "종가": price,
        "시총": total_price
    }
    
    food_stock_list.append(doc)


import openpyxl

wb = openpyxl.Workbook()
ws = wb.active


# 열 제목 추가
ws.append(["날짜", "회사명", "회사코드", "종가", "시총"])


for stock in food_stock_list:
    date = stock['날짜']
    company = stock['회사명']
    company_code = stock['회사코드']
    price = stock['종가']
    total_price = stock['시총']
    
    ws.append([date, company, company_code, price, total_price])


wb.save("crawling_food_company_data.xlsx")
print("엑셀 파일 저장 완료: crawling_food_company_data.xlsx")
    
    

In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
total_pages = 5
daum_datas = []
for page in range(1, total_pages+1):
    search_url = f"https://top.cafe.daum.net/_c21_/search/cafe-table?searchOpt=CAFE_ARTICLE&articleSortType=ACCURACY&q=irp&p={page}"
    driver.get(search_url)
    link_list = [ele.get_attribute('href') for ele in driver.find_elements(By.CLASS_NAME, 'link_tit')]
    for link in link_list:
        sleep(2)
        driver.get(link)
        driver.switch_to.frame("down")
        daum_datas.append(
            {
                'title':driver.find_element(By.CLASS_NAME, "article_title").text,
                'link':link,
                'content' :driver.find_element(By.ID, "user_contents").text
            }
        )

In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
from time import sleep

# 옵션 설정
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

# 크롤 대상 페이지 수
total_pages = 5
daum_datas = []

for page in range(1, total_pages + 1):
    search_url = f"https://top.cafe.daum.net/_c21_/search/cafe-table?searchOpt=CAFE_ARTICLE&articleSortType=ACCURACY&q=irp&p={page}"
    driver.get(search_url)
    sleep(2)

    link_list = [ele.get_attribute("href") for ele in driver.find_elements(By.CLASS_NAME, "link_tit")]

    for link in link_list:
        try:
            driver.get(link)
            sleep(2)

            # iframe으로 변경
            driver.switch_to.frame("down")

            title = driver.find_element(By.CLASS_NAME, "article_title").text
            content = driver.find_element(By.ID, "user_contents").text

            daum_datas.append({
                "title": title,
                "link": link,
                "content": content
            })

            # 원래 페이지로 변경
            driver.switch_to.default_content()
        except Exception as e:
            print(f"❌ 에러 발생: {e}")

# 브라우저 종료
driver.quit()

# ✅ 데이터를 DataFrame으로 변환
df = pd.DataFrame(daum_datas)

# ✅ CSV로 저장
filename = "daum_cafe_data.csv"
df.to_csv(filename, index=False, encoding="utf-8-sig")
print(f"\n✅ 크롤된 {len(df)}개의 게시글이 '{filename}'로 저장되었습니다.")



✅ 크롤된 50개의 게시글이 'daum_cafe_data.csv'로 저장되었습니다.


In [19]:
import pandas as pd

# ✅ 1. 저장된 CSV 파일 로드
csv_filename = "daum_cafe_data.csv"
df = pd.read_csv(csv_filename, encoding="utf-8-sig")

# ✅ 2. TXT 파일로 저장
with open("daum_cafe_data.txt", "w", encoding="utf-8-sig") as f:
    for _, row in df.iterrows():
        title = row.get("title", "")
        link = row.get("link", "")
        content = row.get("content", "")
        f.write(f"제목: {title}\n")
        f.write(f"링크: {link}\n")
        f.write(f"내용:\n{content}\n")
        f.write("="*80 + "\n\n")  # 구분선

print(f"\n✅ 'daum_cafe_data.txt'로 저장 완료되었습니다.")



✅ 'daum_cafe_data.txt'로 저장 완료되었습니다.
